In [95]:
# TAKING PICTURE FROM THE WEBCAM
import cv2

# enables the camera
capture = cv2.VideoCapture(-1)

In [96]:
myPhoto = 'prateek.jpg'
#reads the phot and returns bits and return code
ret, photo = capture.read()

In [97]:
ret

True

In [98]:
photo

array([[[172, 166, 169],
        [172, 166, 169],
        [172, 165, 171],
        ...,
        [126, 121, 124],
        [118, 123, 122],
        [117, 122, 121]],

       [[170, 166, 169],
        [170, 166, 169],
        [171, 168, 170],
        ...,
        [126, 118, 121],
        [120, 120, 120],
        [119, 119, 119]],

       [[171, 168, 170],
        [171, 168, 170],
        [172, 169, 172],
        ...,
        [120, 113, 114],
        [116, 111, 112],
        [115, 110, 111]],

       ...,

       [[100, 102, 108],
        [100, 102, 108],
        [ 99, 103, 111],
        ...,
        [ 73, 140, 184],
        [ 74, 142, 183],
        [ 74, 142, 183]],

       [[ 96, 101, 107],
        [ 98, 102, 108],
        [ 99, 103, 111],
        ...,
        [ 73, 140, 184],
        [ 74, 142, 183],
        [ 74, 142, 183]],

       [[ 98, 102, 108],
        [ 99, 103, 110],
        [ 99, 103, 111],
        ...,
        [ 72, 140, 181],
        [ 73, 141, 182],
        [ 74, 142, 183]]

In [99]:
# writes the bites to file
cv2.imwrite(myPhoto, photo)

True

In [100]:
#release the camera
capture.release()

In [101]:
# UPLOADING TO AMAZON S3 BUCKET
import boto3

s3 = boto3.resource("s3")

In [102]:
# access the bucket
my_bucket = s3.Bucket('prateeknewbucket')

In [103]:
# list all the objects in the bucket
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='prateeknewbucket', key='20180628_202806.jpg')
s3.ObjectSummary(bucket_name='prateeknewbucket', key='rek.jpg')


In [104]:
# uploat the image to the bucket
upload_image = 'rek.jpg'
bucket = 'prateeknewbucket'
s3.Bucket(bucket).upload_file(myPhoto , upload_image)

In [105]:
# creating the client for rekognition
region = 'ap-south-1'
rek = boto3.client('rekognition', region)

In [106]:
# Object and Scene Detection
# fetching the object from s3 and analysing
response = rek.detect_labels(
    Image = {
            'S3Object':{
                'Bucket': bucket,
                'Name': upload_image,
            }
    },
    MaxLabels=10,
    MinConfidence=60
)

In [107]:
response

{'Labels': [{'Name': 'Face',
   'Confidence': 99.7151870727539,
   'Instances': [],
   'Parents': [{'Name': 'Person'}]},
  {'Name': 'Person',
   'Confidence': 99.7151870727539,
   'Instances': [{'BoundingBox': {'Width': 0.8256145715713501,
      'Height': 0.921861469745636,
      'Left': 0.16246390342712402,
      'Top': 0.054043833166360855},
     'Confidence': 99.56011199951172}],
   'Parents': []},
  {'Name': 'Man',
   'Confidence': 92.59078216552734,
   'Instances': [],
   'Parents': [{'Name': 'Person'}]},
  {'Name': 'Smile',
   'Confidence': 83.72515869140625,
   'Instances': [],
   'Parents': [{'Name': 'Face'}, {'Name': 'Person'}]},
  {'Name': 'Portrait',
   'Confidence': 83.64909362792969,
   'Instances': [],
   'Parents': [{'Name': 'Face'}, {'Name': 'Photography'}, {'Name': 'Person'}]},
  {'Name': 'Photography',
   'Confidence': 83.64909362792969,
   'Instances': [],
   'Parents': [{'Name': 'Person'}]},
  {'Name': 'Clothing',
   'Confidence': 73.2695083618164,
   'Instances': [

In [108]:
# printing all the labels above 80% confidence
for i in range(len(response['Labels'])):
    print ( response['Labels'][i]['Name'] )

Face
Person
Man
Smile
Portrait
Photography
Clothing
Selfie
Electronics
Teeth


In [109]:
# Facial Analysis
resfaces = rek.detect_faces(
 Image={
          'S3Object': {
              'Bucket': bucket,
              'Name': upload_image,
          }
      },
   Attributes=['ALL']
    
)

In [110]:
resfaces

{'FaceDetails': [{'BoundingBox': {'Width': 0.2956976890563965,
    'Height': 0.5318981409072876,
    'Left': 0.423196405172348,
    'Top': 0.2149604856967926},
   'AgeRange': {'Low': 22, 'High': 34},
   'Smile': {'Value': True, 'Confidence': 98.96556091308594},
   'Eyeglasses': {'Value': False, 'Confidence': 99.7980728149414},
   'Sunglasses': {'Value': False, 'Confidence': 99.91218566894531},
   'Gender': {'Value': 'Male', 'Confidence': 99.68081665039062},
   'Beard': {'Value': True, 'Confidence': 90.43875122070312},
   'Mustache': {'Value': False, 'Confidence': 78.68651580810547},
   'EyesOpen': {'Value': True, 'Confidence': 99.78585815429688},
   'MouthOpen': {'Value': True, 'Confidence': 99.58349609375},
   'Emotions': [{'Type': 'HAPPY', 'Confidence': 98.62635040283203},
    {'Type': 'SURPRISED', 'Confidence': 0.4535077214241028},
    {'Type': 'FEAR', 'Confidence': 0.26924949884414673},
    {'Type': 'DISGUSTED', 'Confidence': 0.15970763564109802},
    {'Type': 'ANGRY', 'Confidence'

In [111]:
if resfaces['FaceDetails'][0]['Smile']['Value'] == True:
    print("Keep smiling!")
else:
    print("Don't worry!,\nEverything is okay.")

Keep smiling!
